In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from pimpa.trade_models.portfolio import Portfolio
from pimpa.evaluators.ccr_valuation_session import CCR_Valuation_Session
from data.configuration.global_parameters import global_parameters
from pimpa.utils.calendar_utils import transform_dates_to_time_differences

from pimpa.pricing_models.interest_rate_swap_pricer import InterestRateSwapPricer
from pimpa.pricing_models.equity_european_option_pricer import EquityEuropeanOptionPricer
from pimpa.trade_models.interest_rate_swap import InterestRateSwap
from pimpa.trade_models.equity_european_option import EquityEuropeanOption
from pimpa.market_data_objects.market_data_builder import MarketDataBuilder
from pimpa.utils.calendar_utils import generate_simulation_dates_schedule

from data.configuration.global_parameters import global_parameters, calibration_parameters


# Construction of a portfolio based on sourced data

1) Create a portfolio object
2) Check the portfolio object attributes & methods 
3) Check the data that has been sourced (master_ledger, desks, counterparty)

In [2]:
netting_agreement_id = 23
test_portfolio_1 = Portfolio(netting_agreement_id)
print('-----------------------------')
print('BEFORE LOAD:')
print(test_portfolio_1)
print('-----------------------------')
test_portfolio_1.load(global_parameters)
print('AFTER LOAD:')
print(test_portfolio_1)
print('-----------------------------')
print(test_portfolio_1.netting_sets)
print('-----------------------------')

-----------------------------
BEFORE LOAD:
Portfolio with netting_agreement_id: 23
NOT AVAILABLE
-----------------------------
AFTER LOAD:
Portfolio with netting_agreement_id: 23
- including trade_IDs: [1234, 1235, 1236, 3234]
- including netting_sets: ['23_MAIN', '23_SINGLETON_1']
- with underlyings: {'USD_LIBOR_3M_CURVE', 'EUR_LIBOR_3M_CURVE', 'USD_ZERO_YIELD_CURVE', 'EUR_ZERO_YIELD_CURVE', 'EUR_USD_FX_RATE'}
- with 65 valuation points.
AVAILABLE
-----------------------------
{'23_MAIN': [1234, 1235, 3234], '23_SINGLETON_1': [1236]}
-----------------------------


1) Check portfolio trades
2) Check trade object and methods

In [4]:
print(test_portfolio_1.trade_inventory)
print('-----------------------------')
for _, trade in test_portfolio_1.trade_inventory.items():
    print(trade)
    print('-----------------------------')

{1234: <pimpa.trade_models.interest_rate_swap.InterestRateSwap object at 0x0000021E0BD6CE80>, 1235: <pimpa.trade_models.interest_rate_swap.InterestRateSwap object at 0x0000021E6A228E80>, 1236: <pimpa.trade_models.interest_rate_swap.InterestRateSwap object at 0x0000021E0BDBCA60>, 3234: <pimpa.trade_models.interest_rate_swap.InterestRateSwap object at 0x0000021E0BDF2850>}
-----------------------------
Trade with trade_id: 1234
- trade type: IRS
- asset class: IR
- trade currency: USD
- trade underlyings: ['USD_LIBOR_3M_CURVE', 'USD_ZERO_YIELD_CURVE']
- trade attributes keys: ['notional', 'currency', 'floating_rate', 'K', 'payer/receiver', 'first_fixing_date', 'last_fixing_date', 'first_payment_date', 'last_payment_date', 'payments_frequency', 'maturity', 'fixings_schedule', 'payments_schedule'])
- with 41 valuation points.
AVAILABLE
-----------------------------
Trade with trade_id: 1235
- trade type: IRS
- asset class: IR
- trade currency: USD
- trade underlyings: ['USD_LIBOR_3M_CURVE',

1) Check collateral agreements

In [5]:
print('-----------------------------')
print(test_portfolio_1.vm_collateral_agreements)
print('-----------------------------')

-----------------------------
{'VM_1': {'trade_ids': [1234, 1235, 3234], 'contractual_terms':   WE_POST_P WE_RECEIVE_R  T_P  T_R  MTA_P  MTA_R
0       YES          YES    2  1.5    0.5    0.3}}
-----------------------------


# Trade valuation (IRS)

1) Construct a IRS trade with child class InterestRateSwap from trade_id
2) Load the trade

In [3]:
trade_id = 1234
risk_factors = pd.read_csv(global_parameters['prototype_data_paths']['RFs_attributes'] + global_parameters['prototype_data_files']['RFs_attributes']['all_RFs_mapping'])
trade = InterestRateSwap(trade_id)
trade.load(global_parameters, risk_factors)
print('-----------------------------')
print(trade)
print('-----------------------------')

-----------------------------
Trade with trade_id: 1234
- trade type: IRS
- asset class: IR
- trade currency: USD
- trade underlyings: ['USD_LIBOR_3M_CURVE', 'USD_ZERO_YIELD_CURVE']
- trade attributes keys: ['notional', 'currency', 'floating_rate', 'K', 'payer/receiver', 'first_fixing_date', 'last_fixing_date', 'first_payment_date', 'last_payment_date', 'payments_frequency', 'maturity', 'fixings_schedule', 'payments_schedule'])
- with 41 valuation points.
AVAILABLE
-----------------------------


1) Construct an IRS pricer with child class InterestRateSwapPricer
2) Create the risk_factor_objects with class MarketDataBuilder (Notice that we do not calibrate the RFE models but we still need to know which are simulated and which are not)
3) Identify the dependencies for the pricer

In [4]:
pricer = InterestRateSwapPricer('IRS_Pricer')
risk_factor_objects = MarketDataBuilder().get_risk_factors(trade.trade_underlyings, global_parameters)
print('-----------------------------')
print(risk_factor_objects)
print('-----------------------------')
pricer_dependencies = pricer.get_market_dependencies(trade.trade_underlyings, risk_factor_objects,calibration_parameters)
print(pricer_dependencies)
print('-----------------------------')

-----------------------------
{'USD_ZERO_YIELD_CURVE': RiskFactor(name='USD_ZERO_YIELD_CURVE', asset_class='IR', asset_type='DISCOUNT_CURVE', currency='USD', simulated=True, model=<pimpa.scenario_generation.hw1f.HW1F object at 0x00000221CCE3DFD0>, model_name='HW1F', reference=None), 'USD_LIBOR_3M_CURVE': RiskFactor(name='USD_LIBOR_3M_CURVE', asset_class='IR', asset_type='SPREAD_TO_DISCOUNT_CURVE', currency='USD', simulated=False, model=None, model_name='NOT_AVAILABLE', reference='USD_ZERO_YIELD_CURVE')}
-----------------------------
{('historical_fixings', 'USD_ZERO_YIELD_CURVE'), ('Pricing_HW1F_calibration', 'USD_ZERO_YIELD_CURVE'), ('historical_fixings', 'USD_LIBOR_3M_CURVE'), ('spread_to_discount_curve', 'USD_LIBOR_3M_CURVE')}
-----------------------------


1) Source market data based on pricer dependencies and using the folders/files pointers in global_parameters (notice that we skipped RFE models dependencies since we do not use them here)
2) Calibrate the pricer 

In [5]:
market_data = MarketDataBuilder().load_market_data(pricer_dependencies, global_parameters)
print('-----------------------------')
print(market_data)
print('-----------------------------')
pricer.calibrate(market_data, calibration_parameters)
print(pricer)
print('-----------------------------')

-----------------------------
{'historical_fixings': {'USD_ZERO_YIELD_CURVE': 2020-01-01    0.01
2019-12-31    0.02
2019-12-30    0.02
2019-12-29    0.02
2019-12-28    0.02
              ... 
2018-12-18    0.02
2018-12-17    0.02
2018-12-16    0.02
2018-12-15    0.02
2018-12-14    0.02
Name: USD_ZERO_YIELD_CURVE, Length: 384, dtype: float64, 'USD_LIBOR_3M_CURVE': 2020-01-01    0.002
2019-12-31    0.002
2019-12-30    0.002
2019-12-29    0.002
2019-12-28    0.002
              ...  
2018-12-18    0.002
2018-12-17    0.002
2018-12-16    0.002
2018-12-15    0.002
2018-12-14    0.002
Name: USD_LIBOR_3M_CURVE, Length: 384, dtype: float64}, 'Pricing_HW1F_calibration': {'USD_ZERO_YIELD_CURVE': {'alpha': 0.05, 'volatility': 0.01, 'rate_curve': <pimpa.market_data_objects.curve.Curve object at 0x00000221AB4A8520>}}, 'spread_to_discount_curve': {'USD_LIBOR_3M_CURVE': <pimpa.market_data_objects.curve.Curve object at 0x00000221AB4A8E20>}}
-----------------------------
-----------------------------


1) Generate a fictious valuation grid
2) Generate fictious scenarios for the only simulated RF ('USD_ZERO_YIELD_CURVE')
3) Use the IRS pricer object to value the fictious scenarios 

In [6]:
scenarios = {'USD_ZERO_YIELD_CURVE': np.random.normal(loc=0.01, scale=0.005, size=(nr_paths, len(valuation_dates)))}

trade_mtms = pricer.price_single_trade(trade, valuation_dates, scenarios, market_data, global_parameters)starting_date = '2020-01-01'
final_date = '2040-01-01'
valuation_frequency = 'monthly'
nr_paths = 1000
global_parameters['n_paths'] = nr_paths
valuation_dates = generate_simulation_dates_schedule(starting_date, final_date, valuation_frequency, global_parameters)


In [ ]:
SimulatedHW1FCurve(scenarios["USD_ZERO_YIELD_CURVE"][:, 10]).get_value(
                    calibration=self.calibration["USD_ZERO_YIELD_CURVE"], t_date=1, 
                    T_date=[2,3], initial_date=valuation_dates[0], return_log=False)
                

In [8]:
np.mean(trade_mtms, axis = 0)

array([15.31522324, 17.2050098 , 15.46872214, 15.55997716, 15.01034174,
       15.99094727, 15.66894853, 13.42615571, 14.97954444, 14.48652106,
       13.84293018, 14.56618171, 14.12815019, 12.66225505, 12.21375686,
       13.21940683, 12.66924519, 12.58604633, 11.20116676, 11.27690821,
       12.3431918 , 12.86891766, 10.43462802, 10.92870948, 10.61511111,
       10.54816903, 10.30702003, 11.39548256,  9.71670656, 10.47524403,
        8.64981795,  9.38845278,  9.58736571,  8.96993994,  8.83633405,
        8.94915941,  8.88468301,  8.36185167,  8.80964227,  8.18946404,
        7.78979712,  8.16176489,  7.97584169,  6.8808239 ,  7.58694482,
        7.46770752,  7.19150134,  6.95595988,  6.5171926 ,  6.95498075,
        7.01134252,  7.00527561,  6.33620973,  6.47731914,  6.49384023,
        6.35654692,  6.09898169,  7.01341523,  6.40288009,  5.48802717,
        6.29667875,  6.10756515,  5.28705462,  5.50881395,  5.21841716,
        5.15498004,  4.85682307,  5.15124665,  4.94764186,  5.30

# Trade valuation (EUR EQ OPT)

1) Construct a EUR EQ OPT trade with child class EquityEuropeanOption from trade_id
2) Load the trade

In [10]:
trade_id = 2345
risk_factors = pd.read_csv(global_parameters['prototype_data_paths']['RFs_attributes'] + global_parameters['prototype_data_files']['RFs_attributes']['all_RFs_mapping'])
trade = EquityEuropeanOption(trade_id)
trade.load(global_parameters, risk_factors)

print(trade)

Trade with trade_id: 2345
- trade type: EQ_EUR_OPT
- asset class: EQ
- trade currency: USD
- trade underlyings: ['CREDIT_SUISSE_SHARE', 'USD_ZERO_YIELD_CURVE', 'CREDIT_SUISSE_IMPLIED_VOLATILITY_SURFACE']
- trade attributes keys: ['notional', 'currency', 'underlying', 'K', 'put/call', 'long/short', 'maturity'])
- with 0 valuation points.
AVAILABLE


1) Construct an EQ_EUR_OPT pricer with child class EquityEuropeanOptionPricer
2) Create the risk_factor_objects with class MarketDataBuilder (Notice that we do not calibrate the RFE models but we still need to know which are simulated and which are not)
3) Identify the dependencies for the pricer

In [11]:
pricer = EquityEuropeanOptionPricer('EQ_EUR_OPT_Pricer')
risk_factor_objects = MarketDataBuilder().get_risk_factors(trade.trade_underlyings, global_parameters)
pricer_dependencies = pricer.get_market_dependencies(trade.trade_underlyings, risk_factor_objects,calibration_parameters)
print(pricer_dependencies)

{('Pricing_HW1F_calibration', 'USD_ZERO_YIELD_CURVE'), ('historical_fixings', 'USD_ZERO_YIELD_CURVE'), ('equity_implied_volatility_surface', 'CREDIT_SUISSE_IMPLIED_VOLATILITY_SURFACE')}


1) Source market data based on pricer dependencies and using the folders/files pointers in global_parameters (notice that we skipped RFE models dependencies since we do not use them here)
2) Calibrate the pricer 

In [12]:
market_data = MarketDataBuilder().load_market_data(pricer_dependencies, global_parameters)
pricer.calibrate(market_data, calibration_parameters)

print(pricer)

1) Generate a fictious valuation grid
2) Generate fictious scenarios for the simulated RFs ('USD_ZERO_YIELD_CURVE' and 'CREDIT_SUISSE_SHARE')
3) Use the EQ_EUR_OPT pricer object to value the fictious scenarios 

In [13]:
starting_date = '2017-01-01'
final_date = '2022-01-01'
valuation_frequency = 'monthly'
nr_paths = 1000
global_parameters['n_paths'] = nr_paths
valuation_dates = generate_simulation_dates_schedule(starting_date, final_date, valuation_frequency, global_parameters)
scenarios = {'USD_ZERO_YIELD_CURVE': np.random.normal(loc=0.01, scale=0.005, size=(nr_paths, len(valuation_dates))), \
            'CREDIT_SUISSE_SHARE':10*np.random.lognormal(mean=0.0, sigma=0.4, size=(nr_paths, len(valuation_dates)))}
trade_mtms = pricer.price_single_trade(trade, valuation_dates, scenarios, market_data, global_parameters)
print('-----------------------------')
print(scenarios)
print('-----------------------------')
print(trade_mtms)
print('-----------------------------')

-----------------------------
{'USD_ZERO_YIELD_CURVE': array([[ 0.0048981 ,  0.01019672,  0.01551665, ...,  0.00696472,
         0.01384737,  0.01437883],
       [ 0.0135735 ,  0.00742293,  0.00340565, ...,  0.01529514,
         0.01812236,  0.00574278],
       [ 0.00381996,  0.01326294, -0.00197518, ...,  0.00577868,
         0.0078118 ,  0.01533414],
       ...,
       [ 0.0027111 ,  0.01477041,  0.0052666 , ...,  0.00933559,
         0.00128013,  0.01377538],
       [ 0.00983083,  0.01792578,  0.01286283, ...,  0.00416176,
         0.00076046,  0.01627034],
       [ 0.0020398 ,  0.0049396 ,  0.00977386, ...,  0.01468964,
         0.00637951,  0.00945151]]), 'CREDIT_SUISSE_SHARE': array([[12.66295734,  5.82649888,  6.55875174, ..., 18.18868326,
         5.66745959,  8.6328191 ],
       [16.53739486,  7.3021255 ,  8.34038488, ...,  3.3305112 ,
         9.28848849,  5.43055914],
       [14.17032896, 11.31592775, 17.61616548, ...,  8.15968202,
         8.94580711, 10.7063644 ],
       .